## 5.4 Zerlegungsverfahren zur Eigenwertbestimmung

### 5.4.1 Cholesky-Verfahren

In [ ]:
import numpy as np
from scripts.Cholesky import cholesky

**Algorithmus 5.1: Cholesky-Verfahren**

In [ ]:
def cholesky_verfahren(A, k):
    A = A.copy()
    n, m = A.shape
    for l in range(k):
        L = cholesky(A)
        A = L.transpose() @ L
        # for i in range(n):
        #     for j in range(n):
        #         a = 0
        #         for k in range(max(i, j), n):
        #             a += L[k, i] * L[k, j]
        #         A[i, j] = a
        # print(f'A_{l} =\n{A}\n')
    return A

*Ergänzende Einzelheiten zum Code*
- Wir verwenden hier das direkte Matrix-Matrix-Produkt von `numpy`, da dies wesentlich effizienter ist als zwei verschachtelte Python-Schleifen.
- Um die Geschwindigkeit von einer jupyter-Zelle zu messen, können Sie einen sogenannten "cell magic" Befehl verwenden. Wenn Sie in der ersten Zeile einer Zelle `%%timit` schreiben, dann wird die Zelle wiederholt ausgeführt und die Zeit gemessen, um die durchschnittliche Rechenzeit zu bestimmen. Probieren Sie es mit der nächsten Zelle aus, um den Geschwindigkeitsunterschied zwischen den Schleifen und dem Matrix-Matrix-Produkt festzustellen.

#### Beispiel 5.20 (Cholesky-Verfahren)
Wir betrachten die Matrix
$$A = \begin{pmatrix}3&-1&0&1\\-1&3&1&1\\ 0&1&3&0\\1&1&0&3\end{pmatrix}.$$
Mit $A_0:=A$ führen wir einige Schritte des Cholesky-Verfahrens mit unserem
Python-Code durch.

In [ ]:
A = np.array([[3, -1, 0, 1],
              [-1, 3, 1, 1],
              [0, 1, 3, 0],
              [1, 1, 0, 3]], dtype=np.double)
A_chol = cholesky_verfahren(A, 50)

Die Eigenwerte sind damit approximiert durch

In [ ]:
lam = np.flip(np.diag(A_chol))
print(lam)

Damit ergeben sich die relativen Fehler

In [ ]:
lam_ex = np.linalg.eig(A)[0]
print(np.abs(lam - lam_ex) / lam_ex)

### 5.4.2 QR-Verfahren

Wir implementieren das QR-Verfahren mithilfe unserer Implementierung des QR-Verfahrens mit Givens-Rotationen.

In [ ]:
from scripts.Givens import qr_givens

In [ ]:
def qr_verfahren(A, k):
    A = A.copy()
    
    for l in range(k):
        QT = qr_givens(A)
        A = A @ QT.transpose()
    return A

#### Beispiel 5.22 (QR-Verfahren)

Wir wenden das QR-Verfahren mit 25 Schritten auf dieselbe Matrix an.

In [ ]:
A_neu = qr_verfahren(A, 25)
lam_qr = np.flip(np.diag(A_neu))
print(lam_qr)

Damit ergeben sich die Fehler

In [ ]:
print(np.abs(lam_qr - lam_ex) / lam_ex)

Nach der Hälfte der Schritte wurden also die Eigenwerte auf die gleiche Genauigkeit bestimmt.

#### Beispiel 5.24 (QR-Verfahren mit Shift)

Wir implementieren das QR-Verfahren mit Shift und wenden dabei in jedem Schritt die Approximation des größten Eigenwertes als Shift an. 

In [ ]:
def qr_verfahren_mit_shift(A, k):
    A = A.copy()
    Id = np.identity(A.shape[0], dtype=A.dtype)
    for l in range(k):
        mu = np.amax(np.diag(A))
        A[:,:] -= mu * Id
        QT = qr_givens(A)
        A[:,:] = A @ QT.transpose() + mu * Id
#         print(f'mu_0 = {mu}')
#         print(f'A_{l} =\n{A}\n')
    return A

Wenden wir dies auf dieselbe Matrix mit nur 6 Schritten an, erhalten wir

In [ ]:
A_neu = qr_verfahren_mit_shift(A, 6)
lam_qrs = np.diag(A_neu)
print(lam_qr)

Damit ergeben sich die Fehler

In [ ]:
print(np.abs(lam_qrs - lam_ex) / lam_ex)

Also erhalten wir wieder einen maximalen relativen Fehler von $0.3\%$.